In [21]:
import json
import os
import pandas as pd
from google.oauth2 import service_account
from googleapiclient.discovery import build

In [22]:
SERVICE_ACCOUNT_FILE = 'service-key.json'
SCOPES = ['https://www.googleapis.com/auth/spreadsheets']

SPREADSHEET_ID = '1zA7EzmwMVJzWOrOHG7VlNnnBoLuQGcKD54wgBuivetY'
MAIN_RANGE = 'Respostas do formulário!B1:L1000'
INDS_RANGE = 'IndustryMapper!A1:B100'
FIX_IND_RANGE = 'fix_industries!A1:M600'
MASTER_RANGE = 'master!A1:M600'
CAT_RANGE = 'categories!A1:Z100'

In [23]:
def get_service_client():
    # Authenticate using the service account
    credentials = service_account.Credentials.from_service_account_file(
        SERVICE_ACCOUNT_FILE, scopes=SCOPES
    )
    
    # Initialize the Google Sheets API service
    service = build('sheets', 'v4', credentials=credentials)

    return service

In [24]:
def read_spreadsheet(service_client, range):
    result = service_client.spreadsheets().values().get(spreadsheetId=SPREADSHEET_ID, range=range).execute()
    values = result.get('values', [])
    df = pd.DataFrame(values[1:], columns=values[0])
    return df

In [25]:
def clear_spreadsheet(service_client, range):
    return service_client.spreadsheets().values().clear(
        spreadsheetId=SPREADSHEET_ID,
        range=range
    ).execute()

In [26]:
def append_spreadsheet(service_client, range, values):
    return service_client.spreadsheets().values().append(
        spreadsheetId=SPREADSHEET_ID, 
        range=range,
        valueInputOption='RAW',
        body={"values": values}
    ).execute()

In [27]:
def df_to_list(df):
    return [df.columns.tolist()] + df.values.tolist()

In [28]:
service = get_service_client()

### Standardize industry

In [29]:
main = read_spreadsheet(service, MAIN_RANGE)
inds = read_spreadsheet(service, INDS_RANGE)

In [30]:
# Clean incorrect industries
fixed_industries = main.merge(inds, on="client_name", suffixes=("", "_clean"), how="left")
fixed_industries["client_industry"] = fixed_industries.client_industry_clean.fillna(fixed_industries.client_industry)
fixed_industries.drop(columns=["client_industry_clean"], inplace=True)

In [31]:
clear_spreadsheet(service, FIX_IND_RANGE)
append_spreadsheet(service, FIX_IND_RANGE, df_to_list(fixed_industries));

### Map categories

In [32]:
order = [
    "client",
    "industry",
    
    "project",
    "description",
    "dt_start",
    
    "channel", 
    "product",
    "design",
    "goal",

    "complexity"
]

In [33]:
# Single-value categories
industryMapper = {
    "Serviços e Tecnologia": "servicos-e-tecnologia",
    "Educação": "educacao",
    "Financeiro": "financeiro",
    "ONGs e Orgs sociais e ambientais": "ongs-orgs",
    "Consultorias": "consultorias",
    "Comunicação e publicidade": "comunicacao",
    "Jornalismo e editorial": "jornalismo",
    "Pesquisa e tendências": "pesquisa",
    
    'Bens de consumo': 'outros',
    "Automotivo": 'outros',
    "Imobiliário": 'outros',
    "Varejo": 'outros',
    'Saúde': 'outros',
}

channelMapper = {
    "midia digital": "digital",
    "midia impressa": "impressa",
    "consultoria_servico": "consultoria"
}


# Multiple-values categories

goalMapper = {
    "educacional": "educacional",
    "informacional": "informacional",
    "jornalistico editorial": "jornalistico-editorial",
    "impacto positivo": "impacto-positivo",
    "institucional": "institucional"
    
}

productMapper = {
    "video": "video",
    "infografico": "infografico",
    "publicacao": "publicacao",
    "apresentacao": "apresentacao",
    "site\x03editorial": "site-editorial",
    "site editorial": "site-editorial",
    "site\x03institucional": "site-institucional",
    "site institucional": "site-institucional",
    "relatorios": "relatorios",
    "outras\x03interfaces digitais": "outras-interfaces",
    "outras interfaces digitais": "outras-interfaces",
}

designMapper = {
    'ilustracao': "ilustracao",
    'infografia': "infografia",
    'motion graphics': "motion-graphics",
    'editorial': "editorial",
    "user interface": "user-interface",
    'datavis': "datavis",
    'design de servicos': "design-de-servicos",
}

In [34]:
def map_categories(series, mapper, multivalues=False):
    if (multivalues):
        return series.str.split(", ").apply(lambda arr: [ mapper[val] for val in arr if val != ""]).astype(str)
    else:
        return series.apply(lambda val: mapper[val] if val != "" else None)

In [35]:
to_map = read_spreadsheet(service, FIX_IND_RANGE)

# Drop not-needed columns and rename the remaining ones
to_map.drop(columns=[ "tem mais de uma entrega" ], inplace=True, errors='ignore')

to_map.rename(columns={
    "complexity_level": "complexity",
    "assignment_name": "description",
    "tipo de mídia": "channel",
    "product_type (selecionar apenas 1)": "product",
    "design_category": "design",
    "client_industry": "industry",
    "client_name": "client",
    "project_name": "project"
}, inplace=True)

to_map["goal"] = map_categories(to_map["goal"], goalMapper, True)
to_map["product"] = map_categories(to_map["product"], productMapper, True)
to_map["design"] = map_categories(to_map["design"], designMapper, True)
to_map["industry"] = map_categories(to_map["industry"], industryMapper)
to_map["channel"] = map_categories(to_map["channel"], channelMapper)

to_map = to_map[order]

In [36]:
clear_spreadsheet(service, MASTER_RANGE)
append_spreadsheet(service, MASTER_RANGE, df_to_list(to_map));

### Make categories tab

In [138]:
from io import StringIO

In [139]:
cat_string = '''
type,name,data
goal,educacional,"{""color"":""#aed8ff""}"
goal,informacional,"{""color"":""#b4ffc5""}"
goal,impacto-positivo,"{""color"":""#fff1b6""}"
goal,jornalistico-editorial,"{""color"":""#ffe0be""}"
goal,institucional,"{""color"":""#b1f5ff""}"
industry,servicos-e-tecnologia,{}
industry,educacao,{}
industry,financeiro,{}
industry,ongs-orgs,{}
industry,consultorias,{}
industry,comunicacao,{}
industry,jornalismo,{}
industry,pesquisa,{}
industry,outros,{}
channel,digital,{}
channel,impresso,{}
channel,consultoria,{}
product,site-editorial,{}
product,outras-interfaces,{}
product,relatorios,{}
product,apresentacao,{}
product,infografico,{}
product,publicacao,{}
product,site-institucional,{}
product,video,{}
design,user-interface,{}
design,datavis,{}
design,ilustracao,{}
design,design-de-servicos,{}
design,editorial,{}
design,infografia,{}
design,motion-graphics,{}
'''

In [140]:
categories = pd.read_csv(StringIO(cat_string))

clear_spreadsheet(service, CAT_RANGE)
append_spreadsheet(service, CAT_RANGE, df_to_list(categories));